In [ ]:
import pandas as pd

import gretel_trainer.b2 as b2

# Gretel dataset
repo = b2.GretelDatasetRepo()
iris = repo.get_dataset("iris")

# Custom datasets (DF, CSV)
bikes = b2.make_dataset(pd.read_csv("~/Downloads/bikebuying.csv"), datatype="tabular", name="bikes")
tiny = b2.make_dataset("~/Downloads/tiny.csv", datatype=b2.Datatype.tabular, name="tiny")

In [ ]:
# SDK

sdk_comp = b2.compare(
    datasets=[iris, bikes, tiny], 
    models=[b2.GretelAmplify], 
    project_display_name="b2sdk",
    working_dir="b2sdk",
    trainer=False,
)

In [ ]:
sdk_comp.results

In [ ]:
# Trainer

trainer_comp = b2.compare(
    datasets=[iris, bikes, tiny], 
    models=[b2.GretelAmplify], 
    project_display_name="b2trainer",
    working_dir="b2trainer",
    trainer=True,
)

In [ ]:
trainer_comp.results